In [1]:
#import package 哈哈哈
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
#这个是测试数据 选择开发者出现频率为10-13的人
new_file_path = 'dataprocessed_10_to_13.csv'
new_bug_raw = pd.read_csv(new_file_path, encoding='latin-1')
label_encoder = LabelEncoder()
new_bug_raw['who_encoded'] = label_encoder.fit_transform(new_bug_raw['who'])



In [51]:
# # 确保使用你提供的特定参数来读取CSV文件
# file_path = 'bug_raw.csv'  # 确保这是正确的文件路径
# bug_raw = pd.read_csv(file_path,
#                       sep='@@,,@@',  # 使用特定的分隔符
#                       engine='python',  # 使用Python引擎
#                       encoding='latin-1')  # 使用latin-1编码

# # 对'who'列进行编码
# label_encoder = LabelEncoder()
# bug_raw['who_encoded'] = label_encoder.fit_transform(bug_raw['who'])

attention_mask是一个在自然语言处理（NLP）模型，特别是在使用Transformer架构（如XLNet、BERT等）时常用的概念。它是一个与输入数据（如文本序列）形状相同的二进制（0或1）序列，用于指示模型哪些部分的输入是实际数据，哪些部分是为了达到固定长度而添加的填充（padding）。
在处理文本数据时，由于不同文本的长度可能不同，为了能够将这些文本作为一个批次输入模型，需要将它们填充到相同的长度。这时，attention_mask就发挥了作用：
值为1的位置告诉模型，这部分是文本的实际内容，需要关注。
值为0的位置告诉模型，这部分是填充的内容，不应该对模型的输出产生影响。
例如，假设我们有一个长度为5的文本序列，经过填充后长度变为7，那么对应的attention_mask可能如下所示：
在这个例子中，attention_mask的前三个值为1，表示"Hello", "world", "!" ,"0" ,"0"是序列的实际内容；后两个值为0，表示这两个位置是填充的内容，模型在处理时应该忽略它们的影响。
使用attention_mask的好处是，它允许模型动态地对每个序列的实际长度进行编码，从而在保持批处理效率的同时，确保模型的输出不会被无关的填充内容所干扰。这对于保持模型性能尤为重要。

In [3]:
import torch
from torch.utils.data import Dataset

class BugReportDataset(Dataset):
    def __init__(self, bug_ids, descriptions, summaries, labels, tokenizer):
        self.bug_ids = bug_ids
        self.descriptions = descriptions
        self.summaries = summaries
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        bug_id = self.bug_ids[idx]
        description = self.descriptions[idx]
        summary = self.summaries[idx]
        label = self.labels[idx]
        
        # 将bug_id转换为字符串，并与summary拼接
        text = f"Bug ID {str(bug_id)}: {str(summary)}"
        
        encoded_input = self.tokenizer(text, truncation=True, max_length=512)
        
        return {
            'input_ids': encoded_input['input_ids'],
            'attention_mask': encoded_input['attention_mask'],
            'labels': label
        }

In [8]:
# 设置XLNet的tokenizer
model_folder_path = './xlnet_cased_L-12_H-768_A-12/'
tokenizer = XLNetTokenizer.from_pretrained(model_folder_path)

In [5]:
# 准备训练和测试数据
# 确保在分割数据时也包括了bug_id列
# 这行代码是使用train_test_split函数从sklearn.model_selection模块来分割数据集为训练集和验证集。具体来说，它做了以下几件事情：
# 1. 输入数据：new_bug_raw[['bug_id', 'description', 'summary']]表示从new_bug_raw这个DataFrame中选取bug_id、description和summary这三列作为特征数据。new_bug_raw['who_encoded']则是标签数据，即每个缺陷报告分配给的开发者，这里假设已经通过LabelEncoder转换为整数编码。
# 2. 分割数据集：train_test_split函数将特征数据和标签数据分割为训练集和验证集。test_size=0.2参数指定了20%的数据将被用作验证集，剩下的80%用作训练集。
# 3. 输出：函数返回四个部分：X_train和y_train是训练集的特征和标签，X_val和y_val是验证集的特征和标签。
# 这个过程是机器学习和深度学习项目中常见的数据准备步骤，目的是确保模型在独立的数据集上进行训练和评估，以避免过拟合并评估模型的泛化能力。
X_train, X_val, y_train, y_val = train_test_split(new_bug_raw[['bug_id', 'description', 'summary']], new_bug_raw['who_encoded'], test_size=0.2)

# 在初始化BugReportDataset实例时，也传入bug_ids
train_dataset = BugReportDataset(X_train['bug_id'].values, X_train['description'].values, X_train['summary'].values, y_train.values, tokenizer)
val_dataset = BugReportDataset(X_val['bug_id'].values, X_val['description'].values, X_val['summary'].values, y_val.values, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

# 初始化模型和优化器的代码保持不变
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=len(label_encoder.classes_))
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\ProgramData\anaconda3\envs\django5\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
#方法2才会运行
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]
    labels = torch.tensor([item['labels'] for item in batch], dtype=torch.long)
    
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    
    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'labels': labels
    }

# 使用 collate_fn 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=collate_fn)


In [11]:
#方法二的进阶
import os
import torch
#方法2 进度条版本
from tqdm.auto import tqdm
import warnings

# 忽略特定的警告
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned*")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# 假设checkpoint_path是你保存检查点的路径
checkpoint_path = 'model_checkpoint.pth'

# 检查是否有可用的检查点
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'Resuming training from epoch {start_epoch}')
else:
    start_epoch = 0
    print('Starting training from scratch')
num_epochs = 113
for epoch in range(start_epoch, num_epochs):
    # 进行训练...
    model.train()
    # 将train_loader包装在tqdm进度条中
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        # 在进度条中显示当前批次的损失值
        progress_bar.set_postfix(loss=loss.item())
    # 在每个epoch结束后保存检查点
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    # 简单评估
    model.eval()
    correct = 0
    total = 0
    # 也可以为验证循环添加进度条
    with torch.no_grad():
        val_progress_bar = tqdm(val_loader, desc="Validating")
        for batch in val_progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Accuracy after epoch {epoch + 1}: {100 * correct / total:.2f}%')
    

Starting training from scratch


Epoch 1:   0%|          | 0/154 [00:00<?, ?it/s]

Validating:   0%|          | 0/39 [00:00<?, ?it/s]

Accuracy after epoch 1: 3.25%


Epoch 2:   0%|          | 0/154 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/39 [00:00<?, ?it/s]

Accuracy after epoch 112: 40.91%


Epoch 113:   0%|          | 0/154 [00:00<?, ?it/s]

Validating:   0%|          | 0/39 [00:00<?, ?it/s]

Accuracy after epoch 113: 41.56%


In [57]:
#方法二的进阶
import os
import torch
#方法2 进度条版本
from tqdm.auto import tqdm
import warnings

# 忽略特定的警告
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned*")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# 假设checkpoint_path是你保存检查点的路径
checkpoint_path = 'model_checkpoint.pth'

# 检查是否有可用的检查点
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'Resuming training from epoch {start_epoch}')
else:
    start_epoch = 0
    print('Starting training from scratch')
num_epochs = 3
for epoch in range(start_epoch, num_epochs):
    # 进行训练...
    model.train()
    # 将train_loader包装在tqdm进度条中
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        # 在进度条中显示当前批次的损失值
        progress_bar.set_postfix(loss=loss.item())
    # 在每个epoch结束后保存检查点
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    # 简单评估
    model.eval()
    correct = 0
    total = 0
    # 也可以为验证循环添加进度条
    with torch.no_grad():
        val_progress_bar = tqdm(val_loader, desc="Validating")
        for batch in val_progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f'Accuracy after epoch {epoch + 1}: {100 * correct / total:.2f}%')
    

Starting training from scratch


Epoch 1:   0%|          | 0/39992 [00:00<?, ?it/s]

KeyboardInterrupt: 